# Investment Tracker

## Import das Libs

In [8]:
import os
import pandas as pd

## Transactions

In [155]:
class Transactions:
    
    def __init__(self):
        self.data_raw = None
        self.data = None
    
    
    @staticmethod
    def rename_cols(dataframe):
        dataframe = dataframe.rename(columns={'Entrada/Saída': "FLOW",
                                              'Data': 'DATE',
                                              'Movimentação': 'ACTION_TYPE',
                                              'Produto': "PRODUCT_NAME",
                                              'Instituição': 'BROKER',
                                              'Quantidade': 'QUANTITY',
                                              'Preço unitário': 'PRICE_UNIT',
                                              'Valor da Operação': "VALUE",
                                              'Sigla': 'TICKER',
                                              'Tipo': 'ASSET_TYPE'})
        return dataframe
    
    
    @staticmethod
    def find_ticker(product_name):
    
        ticker = product_name.split(" - ")[0].strip()

        return ticker
    
    
    @staticmethod
    def adjust_product_name(product_name):
    
        product_name = " ".join(product_name.split())
        components = product_name.split(" - ")
        product_name = " ".join(components[1:])

        return product_name
    
    
    @staticmethod
    def type_of_product(ticker):
    
        if any(x in ticker for x in ["33", "34"]):
            return "BDR"
        elif any(x in ticker for x in ["11", "12"]):
            return "FII"
        elif any(x in ticker for x in ["1", "2", "3", "4"]):
            return "ACAO"
        else:
            return 'OTHER'
    
    
    @staticmethod
    def adjust_flow(flow):
    
        if flow.strip() == "Credito":
            return "Entrada"
        else:
            return "Saida"   

    
    def preprocessing(self, log=False):
        
        data_preprocessed = self.rename_cols(self.data_raw)
        data_preprocessed.sort_values(by="DATE", inplace=True)
        data_preprocessed.reset_index(drop=True, inplace=True)
        data_preprocessed['TICKER'] = data_preprocessed.PRODUCT_NAME.apply(self.find_ticker)
        data_preprocessed['PRODUCT_NAME'] = data_preprocessed.PRODUCT_NAME.apply(self.adjust_product_name)
        data_preprocessed['ASSET_TYPE'] = data_preprocessed.TICKER.apply(self.type_of_product)
        data_preprocessed['FLOW'] = data_preprocessed['FLOW'].apply(self.adjust_flow)
        data_preprocessed = data_preprocessed[["FLOW", "DATE", "ACTION_TYPE", "ASSET_TYPE", "TICKER", "PRODUCT_NAME", "BROKER", "QUANTITY", "PRICE_UNIT", "VALUE"]]
        
        if log == True:
            print("DONE: Transaction preprocessing")
        
        self.data = data_preprocessed
            

    def read_database(self, log=False):
        path = os.getcwd()
        files = os.listdir(path + "/databases")
    
        files_excel = [file for file in files if file[-4:] == 'xlsx']
        files_excel
    
        if log == True:
            [print(f) for f in files if f[-4:] == 'xlsx']
            
        database = pd.DataFrame()
        for file in files_excel:
            data = pd.read_excel('databases/' + file,
                                 parse_dates=['Data'],
                                 date_parser=lambda x: pd.to_datetime(x, format='%d/%m/%Y'))
            database = database.append(data)
            
        self.data_raw = database
        

In [156]:
transactions = Transactions()
transactions.read_database(log=True)

movimentacao-2019.xlsx
movimentacao-2020.xlsx
movimentacao-2021.xlsx
movimentacao-2022.xlsx


C:\Users\joaoa\anaconda3\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
C:\Users\joaoa\anaconda3\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [157]:
transactions.preprocessing(log=True)

DONE: Transaction preprocessing


In [153]:
transactions.data_raw.head()

,Entrada/Saída,Data,Movimentação,Produto,Instituição,Quantidade,Preço unitário,Valor da Operação
0,Credito,2019-12-31,Transferência - Liquidação,BIDI4 - BANCO INTER S.A.,EASYNVEST - TITULO CV S/A,104,15.7,1632.8
1,Credito,2019-12-31,Transferência - Liquidação,MGLU3 - MAGAZINE LUIZA S.A.,EASYNVEST - TITULO CV S/A,33,49.37,1629.21
2,Credito,2019-12-31,Transferência - Liquidação,VVAR3 - VIA VAREJO S.A.,EASYNVEST - TITULO CV S/A,130,11.73,1524.9
3,Credito,2019-12-31,Transferência - Liquidação,SPTW11 - SP DOWNTOWN FDO INV IMOB - FII,EASYNVEST - TITULO CV S/A,2,115.99,231.98
0,Credito,2020-12-29,Atualização,VVAR3 - VIA VAREJO S.A. ...,EASYNVEST - TITULO CV S/A,249,-,-


In [154]:
transactions.data.head()

,FLOW,DATE,ACTION_TYPE,ASSET_TYPE,TICKER,PRODUCT_NAME,BROKER,QUANTITY,PRICE_UNIT,VALUE
0,Entrada,2019-12-31,Transferência - Liquidação,ACAO,BIDI4,BANCO INTER S.A.,EASYNVEST - TITULO CV S/A,104,15.7,1632.8
1,Entrada,2019-12-31,Transferência - Liquidação,ACAO,MGLU3,MAGAZINE LUIZA S.A.,EASYNVEST - TITULO CV S/A,33,49.37,1629.21
2,Entrada,2019-12-31,Transferência - Liquidação,ACAO,VVAR3,VIA VAREJO S.A.,EASYNVEST - TITULO CV S/A,130,11.73,1524.9
3,Entrada,2019-12-31,Transferência - Liquidação,FII,SPTW11,SP DOWNTOWN FDO INV IMOB FII,EASYNVEST - TITULO CV S/A,2,115.99,231.98
4,Saida,2020-01-02,Juros Sobre Capital Próprio - Transferido,ACAO,BIDI4,BANCO INTER S.A.,EASYNVEST - TITULO CV S/A,104,0.02,1.6


### Old

Import das Bases

In [2]:
path = os.getcwd()
files = os.listdir(path + "/bases")
[print(f) for f in files if f[-4:] == 'xlsx'];

movimentacao-2019.xlsx
movimentacao-2020.xlsx
movimentacao-2021.xlsx
movimentacao-2022.xlsx


In [3]:
files_excel = [f for f in files if f[-4:] == 'xlsx']
files_excel

['movimentacao-2019.xlsx',
 'movimentacao-2020.xlsx',
 'movimentacao-2021.xlsx',
 'movimentacao-2022.xlsx']

In [28]:
df = pd.DataFrame()

for file in files_excel:
    data = pd.read_excel('bases/' + file,
                         parse_dates=['Data'],
                         date_parser=lambda x: pd.to_datetime(x, format='%d/%m/%Y'))
    df = df.append(data)

C:\Users\joaoa\anaconda3\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
C:\Users\joaoa\anaconda3\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [29]:
df[:20]

,Entrada/Saída,Data,Movimentação,Produto,Instituição,Quantidade,Preço unitário,Valor da Operação
0,Credito,2019-12-31,Transferência - Liquidação,BIDI4 - BANCO INTER S.A.,EASYNVEST - TITULO CV S/A,104,15.7,1632.8
1,Credito,2019-12-31,Transferência - Liquidação,MGLU3 - MAGAZINE LUIZA S.A.,EASYNVEST - TITULO CV S/A,33,49.37,1629.21
2,Credito,2019-12-31,Transferência - Liquidação,VVAR3 - VIA VAREJO S.A.,EASYNVEST - TITULO CV S/A,130,11.73,1524.9
3,Credito,2019-12-31,Transferência - Liquidação,SPTW11 - SP DOWNTOWN FDO INV IMOB - FII,EASYNVEST - TITULO CV S/A,2,115.99,231.98
0,Credito,2020-12-29,Atualização,VVAR3 - VIA VAREJO S.A. ...,EASYNVEST - TITULO CV S/A,249,-,-
1,Credito,2020-12-17,Dividendo,CYRE3 - CYRELA BRAZIL REALTY S.A.EMPREEND E PA...,EASYNVEST - TITULO CV S/A,70,0.26,17.98
2,Credito,2020-12-15,Dividendo,PETR3 - PETROLEO BRASILEIRO S.A. PETROBRAS ...,EASYNVEST - TITULO CV S/A,100,0.23,23.36
3,Credito,2020-12-15,Rendimento,PETR3 - PETROLEO BRASILEIRO S.A. PETROBRAS ...,EASYNVEST - TITULO CV S/A,100,0.01,0.5
4,Credito,2020-12-08,Amortização,SPTW11 - SP DOWNTOWN FDO INV IMOB - FII ...,EASYNVEST - TITULO CV S/A,20,0.54,10.8
5,Credito,2020-12-08,Rendimento,SPTW11 - SP DOWNTOWN FDO INV IMOB - FII ...,EASYNVEST - TITULO CV S/A,20,0.7,14


In [30]:
df.sort_values(by="Data", inplace=True)

In [31]:
df.reset_index(drop=True, inplace=True)

In [33]:
df

,Entrada/Saída,Data,Movimentação,Produto,Instituição,Quantidade,Preço unitário,Valor da Operação
0,Credito,2019-12-31,Transferência - Liquidação,BIDI4 - BANCO INTER S.A.,EASYNVEST - TITULO CV S/A,104,15.7,1632.8
1,Credito,2019-12-31,Transferência - Liquidação,MGLU3 - MAGAZINE LUIZA S.A.,EASYNVEST - TITULO CV S/A,33,49.37,1629.21
2,Credito,2019-12-31,Transferência - Liquidação,VVAR3 - VIA VAREJO S.A.,EASYNVEST - TITULO CV S/A,130,11.73,1524.9
3,Credito,2019-12-31,Transferência - Liquidação,SPTW11 - SP DOWNTOWN FDO INV IMOB - FII,EASYNVEST - TITULO CV S/A,2,115.99,231.98
4,Debito,2020-01-02,Juros Sobre Capital Próprio - Transferido,BIDI4 - BANCO INTER S.A. ...,EASYNVEST - TITULO CV S/A,104,0.02,1.6
...,...,...,...,...,...,...,...,...
258,Credito,2022-05-16,Dividendo,PETR4 - PETROLEO BRASILEIRO S/A PETROBRAS,NU INVEST CORRETORA DE VALORES S.A.,86,2.86,246.05
259,Credito,2022-05-16,Rendimento,PETR3 - PETROLEO BRASILEIRO S/A PETROBRAS,NU INVEST CORRETORA DE VALORES S.A.,100,0.11,8.46
260,Credito,2022-05-16,Dividendo,PETR3 - PETROLEO BRASILEIRO S/A PETROBRAS,NU INVEST CORRETORA DE VALORES S.A.,100,2.86,286.1
261,Credito,2022-05-25,Rendimento,BTLG11 - BTG PACTUAL LOGISTICA FUNDO DE ...,NU INVEST CORRETORA DE VALORES S.A.,16,0.72,11.52


In [34]:
def find_ticker(product_name):
    
    ticker = product_name.split(" - ")[0].strip()
    
    return ticker

In [35]:
df.Produto.iloc[21]

'XPML11 - XP MALLS FDO INV IMOB FII                         '

In [113]:
def adjust_product_name(product_name):
    
    product_name = " ".join(product_name.split())
    components = product_name.split(" - ")
    
    product_name = " ".join(components[1:])
    
    return product_name

In [114]:
adjust_product_name("PETR3        - PETROLEO BRASILEIRO S/A PETROBRAS")

'PETROLEO BRASILEIRO S/A PETROBRAS'

In [38]:
df['Sigla'] = df.Produto.apply(find_ticker)

In [39]:
df['Produto'] = df.Produto.apply(adjust_product_name)

In [40]:
df.head()

,Entrada/Saída,Data,Movimentação,Produto,Instituição,Quantidade,Preço unitário,Valor da Operação,Sigla
0,Credito,2019-12-31,Transferência - Liquidação,BANCO INTER S.A. (BIDI4),EASYNVEST - TITULO CV S/A,104,15.7,1632.8,BIDI4
1,Credito,2019-12-31,Transferência - Liquidação,MAGAZINE LUIZA S.A. (MGLU3),EASYNVEST - TITULO CV S/A,33,49.37,1629.21,MGLU3
2,Credito,2019-12-31,Transferência - Liquidação,VIA VAREJO S.A. (VVAR3),EASYNVEST - TITULO CV S/A,130,11.73,1524.9,VVAR3
3,Credito,2019-12-31,Transferência - Liquidação,SP DOWNTOWN FDO INV IMOB FII (SPTW11),EASYNVEST - TITULO CV S/A,2,115.99,231.98,SPTW11
4,Debito,2020-01-02,Juros Sobre Capital Próprio - Transferido,BANCO INTER S.A. (BIDI4),EASYNVEST - TITULO CV S/A,104,0.02,1.6,BIDI4


In [41]:
df.Sigla.unique()

array(['BIDI4', 'MGLU3', 'VVAR3', 'SPTW11', 'XPML11', 'BTOW3', 'JHSF3',
       'AZUL4', 'PETR3', 'WEGE3', 'ITUB4', 'CYRE3', 'BIDI2', 'BTOW1',
       'B3SA3', 'VALE3', 'SULA11', 'BBDC3', 'TSLA34', 'CSAN3', 'ABEV3',
       'KISU11', 'NVDC34', 'AMZO34', 'XINA11', 'PETR4', 'S1PO34',
       'KISU12', 'BTLG11', 'BPAC3', 'BTLG12', 'NUBR33', 'NFLX34'],
      dtype=object)

In [42]:
def type_of_product(ticker):
    
    if any(x in ticker for x in ["33", "34"]):
        return "BDR"
    elif any(x in ticker for x in ["11", "12"]):
        return "FII"
    elif any(x in ticker for x in ["1", "2", "3", "4"]):
        return "ACAO"
    else:
        return 'OTHER'

In [43]:
df['Tipo'] = df.Sigla.apply(type_of_product)

In [44]:
df.head()

,Entrada/Saída,Data,Movimentação,Produto,Instituição,Quantidade,Preço unitário,Valor da Operação,Sigla,Tipo
0,Credito,2019-12-31,Transferência - Liquidação,BANCO INTER S.A. (BIDI4),EASYNVEST - TITULO CV S/A,104,15.7,1632.8,BIDI4,ACAO
1,Credito,2019-12-31,Transferência - Liquidação,MAGAZINE LUIZA S.A. (MGLU3),EASYNVEST - TITULO CV S/A,33,49.37,1629.21,MGLU3,ACAO
2,Credito,2019-12-31,Transferência - Liquidação,VIA VAREJO S.A. (VVAR3),EASYNVEST - TITULO CV S/A,130,11.73,1524.9,VVAR3,ACAO
3,Credito,2019-12-31,Transferência - Liquidação,SP DOWNTOWN FDO INV IMOB FII (SPTW11),EASYNVEST - TITULO CV S/A,2,115.99,231.98,SPTW11,FII
4,Debito,2020-01-02,Juros Sobre Capital Próprio - Transferido,BANCO INTER S.A. (BIDI4),EASYNVEST - TITULO CV S/A,104,0.02,1.6,BIDI4,ACAO


In [45]:
df['Data'] = pd.to_datetime(df.Data, format="%d-%m-%Y")

In [46]:
df[df['Sigla'] == "CYRE3"].sort_values(by="Data")

,Entrada/Saída,Data,Movimentação,Produto,Instituição,Quantidade,Preço unitário,Valor da Operação,Sigla,Tipo
56,Credito,2020-06-03,Transferência - Liquidação,CYRELA BRAZIL REALTY S.A.EMPREEND E PART (CYRE3),EASYNVEST - TITULO CV S/A,50,17.18,859,CYRE3,ACAO
63,Credito,2020-07-15,Transferência - Liquidação,CYRELA BRAZIL REALTY S.A.EMPREEND E PART (CYRE3),EASYNVEST - TITULO CV S/A,20,27.57,551.4,CYRE3,ACAO
68,Credito,2020-07-28,Transferência - Liquidação,CYRELA BRAZIL REALTY S.A.EMPREEND E PART (CYRE3),EASYNVEST - TITULO CV S/A,30,25.26,757.8,CYRE3,ACAO
113,Debito,2020-11-04,Transferência - Liquidação,CYRELA BRAZIL REALTY S/A EMPREENDIMENTOS E PAR...,NU INVEST CORRETORA DE VALORES S.A.,100,22.74,2274,CYRE3,ACAO
124,Credito,2020-12-17,Dividendo,CYRELA BRAZIL REALTY S.A.EMPREEND E PART (CYRE3),EASYNVEST - TITULO CV S/A,70,0.26,17.98,CYRE3,ACAO
128,Credito,2021-01-07,Transferência - Liquidação,CYRELA BRAZIL REALTY S/A EMPREENDIMENTOS E PAR...,NU INVEST CORRETORA DE VALORES S.A.,100,27.86,2786,CYRE3,ACAO
184,Credito,2021-06-14,Dividendo,CYRELA BRAZIL REALTY S/A EMPREENDIMENTOS E PAR...,EASYNVEST - TITULO CV S/A,100,1.09,108.72,CYRE3,ACAO
220,Debito,2021-11-04,Transferência - Liquidação,CYRELA BRAZIL REALTY S/A EMPREENDIMENTOS E PAR...,NU INVEST CORRETORA DE VALORES S.A.,100,14.53,1453,CYRE3,ACAO


In [47]:
df['Entrada/Saída'].unique()

array(['Credito', 'Debito'], dtype=object)

In [48]:
def adjust_entrada_saida(entrada_saida):
    
    if entrada_saida.strip() == "Credito":
        return "Entrada"
    else:
        return "Saida"

In [49]:
df['Fluxo'] = df['Entrada/Saída'].apply(adjust_entrada_saida)

In [50]:
df

,Entrada/Saída,Data,Movimentação,Produto,Instituição,Quantidade,Preço unitário,Valor da Operação,Sigla,Tipo,Fluxo
0,Credito,2019-12-31,Transferência - Liquidação,BANCO INTER S.A. (BIDI4),EASYNVEST - TITULO CV S/A,104,15.7,1632.8,BIDI4,ACAO,Entrada
1,Credito,2019-12-31,Transferência - Liquidação,MAGAZINE LUIZA S.A. (MGLU3),EASYNVEST - TITULO CV S/A,33,49.37,1629.21,MGLU3,ACAO,Entrada
2,Credito,2019-12-31,Transferência - Liquidação,VIA VAREJO S.A. (VVAR3),EASYNVEST - TITULO CV S/A,130,11.73,1524.9,VVAR3,ACAO,Entrada
3,Credito,2019-12-31,Transferência - Liquidação,SP DOWNTOWN FDO INV IMOB FII (SPTW11),EASYNVEST - TITULO CV S/A,2,115.99,231.98,SPTW11,FII,Entrada
4,Debito,2020-01-02,Juros Sobre Capital Próprio - Transferido,BANCO INTER S.A. (BIDI4),EASYNVEST - TITULO CV S/A,104,0.02,1.6,BIDI4,ACAO,Saida
...,...,...,...,...,...,...,...,...,...,...,...
258,Credito,2022-05-16,Dividendo,PETROLEO BRASILEIRO S/A PETROBRAS (PETR4),NU INVEST CORRETORA DE VALORES S.A.,86,2.86,246.05,PETR4,ACAO,Entrada
259,Credito,2022-05-16,Rendimento,PETROLEO BRASILEIRO S/A PETROBRAS (PETR3),NU INVEST CORRETORA DE VALORES S.A.,100,0.11,8.46,PETR3,ACAO,Entrada
260,Credito,2022-05-16,Dividendo,PETROLEO BRASILEIRO S/A PETROBRAS (PETR3),NU INVEST CORRETORA DE VALORES S.A.,100,2.86,286.1,PETR3,ACAO,Entrada
261,Credito,2022-05-25,Rendimento,BTG PACTUAL LOGISTICA FUNDO DE INVESTIMENTO IM...,NU INVEST CORRETORA DE VALORES S.A.,16,0.72,11.52,BTLG11,FII,Entrada


In [51]:
df = df[['Fluxo', "Data", "Movimentação", "Tipo", "Sigla", "Produto", "Instituição", "Quantidade", "Preço unitário", "Valor da Operação"]]

In [52]:
df.columns

Index(['Fluxo', 'Data', 'Movimentação', 'Tipo', 'Sigla', 'Produto',
       'Instituição', 'Quantidade', 'Preço unitário', 'Valor da Operação'],
      dtype='object')

In [53]:
 df.rename(columns={'Fluxo': "FLOW",
                    'Data': 'DATE',
                    'Movimentação': 'ACTION_TYPE',
                    'Produto': "PRODUCT_NAME",
                    'Instituição': 'BROKER',
                    'Quantidade': 'QUANTITY',
                    'Preço unitário': 'PRICE_UNIT',
                    'Valor da Operação': "VALUE",
                    'Sigla': 'TICKER',
                    'Tipo': 'ASSET_TYPE'}, inplace=True)

C:\Users\joaoa\anaconda3\lib\site-packages\pandas\core\frame.py:5039: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


In [54]:
df.DATE.dtype

dtype('<M8[ns]')

In [55]:
df.head()

,FLOW,DATE,ACTION_TYPE,ASSET_TYPE,TICKER,PRODUCT_NAME,BROKER,QUANTITY,PRICE_UNIT,VALUE
0,Entrada,2019-12-31,Transferência - Liquidação,ACAO,BIDI4,BANCO INTER S.A. (BIDI4),EASYNVEST - TITULO CV S/A,104,15.7,1632.8
1,Entrada,2019-12-31,Transferência - Liquidação,ACAO,MGLU3,MAGAZINE LUIZA S.A. (MGLU3),EASYNVEST - TITULO CV S/A,33,49.37,1629.21
2,Entrada,2019-12-31,Transferência - Liquidação,ACAO,VVAR3,VIA VAREJO S.A. (VVAR3),EASYNVEST - TITULO CV S/A,130,11.73,1524.9
3,Entrada,2019-12-31,Transferência - Liquidação,FII,SPTW11,SP DOWNTOWN FDO INV IMOB FII (SPTW11),EASYNVEST - TITULO CV S/A,2,115.99,231.98
4,Saida,2020-01-02,Juros Sobre Capital Próprio - Transferido,ACAO,BIDI4,BANCO INTER S.A. (BIDI4),EASYNVEST - TITULO CV S/A,104,0.02,1.6


In [56]:
df['DATE'] = pd.to_datetime(df['DATE'])

C:\Users\joaoa\AppData\Local\Temp/ipykernel_20296/2541906660.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['DATE'] = pd.to_datetime(df['DATE'])


In [57]:
print(type(df['DATE'].iloc[0])) 

<class 'pandas._libs.tslibs.timestamps.Timestamp'>


In [58]:
df.sort_values(by=['DATE', 'FLOW'], ascending=[True, False], inplace=True)

C:\Users\joaoa\anaconda3\lib\site-packages\pandas\util\_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


In [59]:
print(type(df['PRICE_UNIT'].iloc[0])) 
print(type(df['VALUE'].iloc[0])) 

<class 'float'>
<class 'float'>


In [60]:
df['PRICE_UNIT'] = pd.to_numeric(df['PRICE_UNIT'], errors='coerce')
df['VALUE'] = pd.to_numeric(df['VALUE'], errors='coerce');

C:\Users\joaoa\AppData\Local\Temp/ipykernel_20296/3741329072.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['PRICE_UNIT'] = pd.to_numeric(df['PRICE_UNIT'], errors='coerce')
C:\Users\joaoa\AppData\Local\Temp/ipykernel_20296/3741329072.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['VALUE'] = pd.to_numeric(df['VALUE'], errors='coerce');


In [61]:
df

,FLOW,DATE,ACTION_TYPE,ASSET_TYPE,TICKER,PRODUCT_NAME,BROKER,QUANTITY,PRICE_UNIT,VALUE
0,Entrada,2019-12-31,Transferência - Liquidação,ACAO,BIDI4,BANCO INTER S.A. (BIDI4),EASYNVEST - TITULO CV S/A,104,15.70,1632.80
1,Entrada,2019-12-31,Transferência - Liquidação,ACAO,MGLU3,MAGAZINE LUIZA S.A. (MGLU3),EASYNVEST - TITULO CV S/A,33,49.37,1629.21
2,Entrada,2019-12-31,Transferência - Liquidação,ACAO,VVAR3,VIA VAREJO S.A. (VVAR3),EASYNVEST - TITULO CV S/A,130,11.73,1524.90
3,Entrada,2019-12-31,Transferência - Liquidação,FII,SPTW11,SP DOWNTOWN FDO INV IMOB FII (SPTW11),EASYNVEST - TITULO CV S/A,2,115.99,231.98
4,Saida,2020-01-02,Juros Sobre Capital Próprio - Transferido,ACAO,BIDI4,BANCO INTER S.A. (BIDI4),EASYNVEST - TITULO CV S/A,104,0.02,1.60
...,...,...,...,...,...,...,...,...,...,...
258,Entrada,2022-05-16,Dividendo,ACAO,PETR4,PETROLEO BRASILEIRO S/A PETROBRAS (PETR4),NU INVEST CORRETORA DE VALORES S.A.,86,2.86,246.05
259,Entrada,2022-05-16,Rendimento,ACAO,PETR3,PETROLEO BRASILEIRO S/A PETROBRAS (PETR3),NU INVEST CORRETORA DE VALORES S.A.,100,0.11,8.46
260,Entrada,2022-05-16,Dividendo,ACAO,PETR3,PETROLEO BRASILEIRO S/A PETROBRAS (PETR3),NU INVEST CORRETORA DE VALORES S.A.,100,2.86,286.10
261,Entrada,2022-05-25,Rendimento,FII,BTLG11,BTG PACTUAL LOGISTICA FUNDO DE INVESTIMENTO IM...,NU INVEST CORRETORA DE VALORES S.A.,16,0.72,11.52


In [62]:
df.ACTION_TYPE.unique()

array(['Transferência - Liquidação',
       'Juros Sobre Capital Próprio - Transferido',
       'Juros Sobre Capital Próprio', 'Amortização', 'Rendimento',
       'Transferência', 'Direito de Subscrição', 'Dividendo',
       'Direitos de Subscrição - Não Exercido',
       'Cessão de Direitos - Solicitada', 'Cessão de Direitos',
       'Desdobro', 'Atualização'], dtype=object)

## Asset

In [168]:
class Asset:
    
    def __init__(self, ticker, product_name, asset_type):
        self.ticker = ticker
        self.product_name = product_name
        self.asset_type = asset_type
        self.quantity = 0
        self.value = 0
        self.price_unit = 0
        #self.transaction_history = dataframe
        #self.dividendo_history = dataframe
        self.income = 0
        
    def atualiza_price_unit(self):
        
        if self.quantity == 0:
            return None
        
        else:
            self.price_unit = self.value / self.quantity
            return self.value / self.quantity
        

    def set_quantity(self, quantidade):
        self.quantity = quantidade

        
    def set_value(self, valor):
        self.value = valor
        
    def set_income(self, income_value):
        self.income = income_value
        
        
    def buy(self, quantidade_compra, valor_compra):
        
        quantidade_atual = self.quantity 
        quantidade_final = quantidade_atual + quantidade_compra
        
        valor_atual = self.value 
        valor_final = valor_atual + valor_compra
        
        self.set_quantity(quantidade_final)
        self.set_value(valor_final)
        self.atualiza_price_unit()
        
        
    def sell(self, quantidade_venda, valor_venda):
        
        quantidade_atual = self.quantity 
        quantidade_final = quantidade_atual - quantidade_venda
        
#         valor_atual = self.value 
#         valor_final = valor_atual - valor_venda
        
        if quantidade_final < 0:
            raise Exception('Você tentou vendar uma quantidade maior do que possui para o ativo:', self.ticker)
            
#         if valor_final < 0:
#             raise Exception('Você tentou vendar um valor maior do que possui para o ativo:', self.ticker)    
        
        
        self.set_quantity(quantidade_final)
#         self.set_value(valor_final)
        self.atualiza_price_unit()    
    
    
    
    def split(self, quantidade_split, method='add'):
        
        if method == 'add':
            quantidade_atual = self.quantity 
            quantidade_final = quantidade_atual + quantidade_split
            
        elif method == 'split':
            quantidade_atual = self.quantity
            quantidade_final = quantidade_atual * quantidade_split
            
        self.set_quantity(quantidade_final)
        self.atualiza_price_unit()
        
        
    def income_distribution(self, income_received):
        
        total_asset_income = self.income + income_received
        self.set_income(total_asset_income)
        

        

    def imprime_ativo(self):
        
        print("Ticker:", self.ticker)
        print("Nome do produto:", self.product_name)
        print("Tipo:", self.asset_type)
        print("Quantidade:", self.quantity)
        print("Valor:", self.value)
        print("Preço médio: {:.2f}".format(self.price_unit))
        print("Rendimentos: {:.2f}".format(self.income))
    

### Testing - Asset class

In [169]:
# Cria um exemplo de ativo
teste = Asset("MGLU3", "MAGAZINE LUIZA S.A. (MGLU3)", "ACAO")

In [171]:
teste.imprime_ativo()

Ticker: MGLU3
Nome do produto: MAGAZINE LUIZA S.A. (MGLU3)
Tipo: ACAO
Quantidade: 0
Valor: 0
Preço médio: 0.00
Rendimentos: 0.00


In [176]:
teste.imprime_ativo()

Ticker: MGLU3
Nome do produto: MAGAZINE LUIZA S.A. (MGLU3)
Tipo: ACAO
Quantidade: 200
Valor: 400
Preço médio: 2.00
Rendimentos: 0.00


In [172]:
teste.set_quantity(200)

In [173]:
teste.set_value(400)

In [175]:
teste.atualiza_price_unit()

2.0

In [80]:
teste.imprime_ativo()

Ticker: MGLU3
Nome do produto: MAGAZINE LUIZA S.A. (MGLU3)
Tipo: ACAO
Quantidade: 200
Valor: 400
Preço médio: 2.00


In [81]:
teste.buy(100, 50)

In [82]:
teste.imprime_ativo()

Ticker: MGLU3
Nome do produto: MAGAZINE LUIZA S.A. (MGLU3)
Tipo: ACAO
Quantidade: 300
Valor: 450
Preço médio: 1.50


In [83]:
teste.sell(200, 150)

In [84]:
teste.imprime_ativo()

Ticker: MGLU3
Nome do produto: MAGAZINE LUIZA S.A. (MGLU3)
Tipo: ACAO
Quantidade: 100
Valor: 450
Preço médio: 4.50


In [85]:
teste.split(400)

In [86]:
teste.imprime_ativo()

Ticker: MGLU3
Nome do produto: MAGAZINE LUIZA S.A. (MGLU3)
Tipo: ACAO
Quantidade: 500
Valor: 450
Preço médio: 0.90


## Wallet

In [162]:
class Wallet:

    def __init__(self, name):
        self.name = name
        self.asset_list = []

        
    def asset_exist(self, asset_ticker_searched):      
        for asset in self.asset_list:
            if asset.ticker == asset_ticker_searched:
                return True
        return False
            
        
    def add_asset(self, ticker, product_name, asset_type, log=False):
        
        if self.asset_exist(ticker):
            if log:
                print("{} alredy exist".format(ticker))
            pass
           
        else:
            created_asset = Asset(ticker, product_name, asset_type)
            self.asset_list.append(created_asset)        
            if log:
                print("{} added".format(ticker))
          
        
    def find_asset(self, find_ticker):
        for asset in self.asset_list:
            if asset.ticker == find_ticker:
                return asset
                break            
        

    def print_assets(self):        
        for asset in self.asset_list:
            asset.imprime_ativo()
            print("-"*60)
     
    
    def count_assets(self):
        return len(self.asset_list)

    

            
#     def create(self, ativo)


In [163]:
wallet = Wallet('my wallet')

In [164]:
wallet.add_asset('TEST12', 'Nome do produto', 'TIPO', log=True)

NameError: name 'Asset' is not defined

In [69]:
wallet.print_assets()

In [70]:
wallet.asset_exist("TEST12")

False

In [71]:
wallet.find_asset("TEST12")

In [72]:
wallet.count_assets()

0

## Transaction

In [103]:
class Transaction:
    
    def __init__(self, flow, date, action, asset_type, asset_ticker, asset_name, broker, quantity, value, average_price):
        self.flow = flow
        self.date = date
        self.action = action
        self.asset_type = asset_type
        self.asset_ticker = asset_ticker
        self.asset_name = asset_name
        self.broker = broker
        self.quantity = quantity
        self.value = value
        self.average_price = average_price
        

,FLOW,DATE,ACTION_TYPE,ASSET_TYPE,TICKER,PRODUCT_NAME,BROKER,QUANTITY,PRICE_UNIT,VALUE
0,Entrada,2019-12-31,Transferência - Liquidação,ACAO,BIDI4,BANCO INTER S.A. (BIDI4),EASYNVEST - TITULO CV S/A,104,15.70,1632.80
1,Entrada,2019-12-31,Transferência - Liquidação,ACAO,MGLU3,MAGAZINE LUIZA S.A. (MGLU3),EASYNVEST - TITULO CV S/A,33,49.37,1629.21
2,Entrada,2019-12-31,Transferência - Liquidação,ACAO,VVAR3,VIA VAREJO S.A. (VVAR3),EASYNVEST - TITULO CV S/A,130,11.73,1524.90
3,Entrada,2019-12-31,Transferência - Liquidação,FII,SPTW11,SP DOWNTOWN FDO INV IMOB FII (SPTW11),EASYNVEST - TITULO CV S/A,2,115.99,231.98
4,Saida,2020-01-02,Juros Sobre Capital Próprio - Transferido,ACAO,BIDI4,BANCO INTER S.A. (BIDI4),EASYNVEST - TITULO CV S/A,104,0.02,1.60
...,...,...,...,...,...,...,...,...,...,...
258,Entrada,2022-05-16,Dividendo,ACAO,PETR4,PETROLEO BRASILEIRO S/A PETROBRAS (PETR4),NU INVEST CORRETORA DE VALORES S.A.,86,2.86,246.05
259,Entrada,2022-05-16,Rendimento,ACAO,PETR3,PETROLEO BRASILEIRO S/A PETROBRAS (PETR3),NU INVEST CORRETORA DE VALORES S.A.,100,0.11,8.46
260,Entrada,2022-05-16,Dividendo,ACAO,PETR3,PETROLEO BRASILEIRO S/A PETROBRAS (PETR3),NU INVEST CORRETORA DE VALORES S.A.,100,2.86,286.10
261,Entrada,2022-05-25,Rendimento,FII,BTLG11,BTG PACTUAL LOGISTICA FUNDO DE INVESTIMENTO IM...,NU INVEST CORRETORA DE VALORES S.A.,16,0.72,11.52


## Tracker

In [101]:
imprime = True

# create wallet
wallet = wallet("wallet Teste")

for index, row in df.iterrows():
    
#     if imprime:
#         print(row['TICKER'], row["QUANTITY"], row["VALUE"], row['FLOW'], row['ACTION_TYPE'])
    
    if row["ACTION_TYPE"] == "Transferência - Liquidação":
    
        # Avalia se o ativo existe na wallet, se não, cria o ativo
        if not wallet.asset_exist(row['TICKER']):
            wallet.add_asset(row['TICKER'], row['PRODUCT_NAME'], row['ASSET_TYPE'], feedback=imprime)

        if row['FLOW'] == "Entrada":
            asset = wallet.find_asset(row["TICKER"])
            asset.buy(row["QUANTITY"], row["VALUE"])

            print("comprou", row["TICKER"])


        if row['FLOW'] == "Saida":
            asset = wallet.find_asset(row["TICKER"])
            asset.sell(row["QUANTITY"], row["VALUE"])

            print("vendeu", row["TICKER"])
        
    elif row["ACTION_TYPE"] == "Desdobro":
        asset = wallet.find_asset(row["TICKER"])
        asset.split(row["QUANTITY"])
        print("desdobrou", row["TICKER"])
        
    elif row["ACTION_TYPE"] in ["Dividendo", "Rendimento"]:
        asset = wallet.find_asset(row["TICKER"])
        asset.income_distribution(row["VALUE"])
        print("DIVIDENDO")
    

    else:
        continue
    
    

BIDI4 added
comprou BIDI4
MGLU3 added
comprou MGLU3
VVAR3 added
comprou VVAR3
SPTW11 added
comprou SPTW11
DIVIDENDO
XPML11 added
comprou XPML11
comprou SPTW11
BTOW3 added
comprou BTOW3
comprou VVAR3
DIVIDENDO
vendeu VVAR3
comprou BTOW3
comprou VVAR3
comprou MGLU3
DIVIDENDO
comprou MGLU3
DIVIDENDO
comprou MGLU3
vendeu BIDI4
JHSF3 added
comprou JHSF3
comprou MGLU3
comprou VVAR3
comprou MGLU3
DIVIDENDO
comprou VVAR3
AZUL4 added
comprou AZUL4
comprou VVAR3
comprou MGLU3
DIVIDENDO
vendeu AZUL4
vendeu MGLU3
comprou MGLU3
vendeu VVAR3
vendeu MGLU3
PETR3 added
comprou PETR3
DIVIDENDO
WEGE3 added
comprou WEGE3
comprou VVAR3
ITUB4 added
comprou ITUB4
CYRE3 added
comprou CYRE3
DIVIDENDO
DIVIDENDO
DIVIDENDO
comprou CYRE3
comprou SPTW11
DIVIDENDO
comprou ITUB4
comprou CYRE3
comprou MGLU3
comprou ITUB4
DIVIDENDO
vendeu JHSF3
B3SA3 added
comprou B3SA3
VALE3 added
comprou VALE3
DIVIDENDO
DIVIDENDO
DIVIDENDO
comprou VVAR3
DIVIDENDO
BBDC3 added
comprou BBDC3
comprou SPTW11
DIVIDENDO
comprou MGLU3
DIVIDE

In [89]:
df[df.TICKER == "BTLG11"]

,FLOW,DATE,ACTION_TYPE,ASSET_TYPE,TICKER,PRODUCT_NAME,BROKER,QUANTITY,PRICE_UNIT,VALUE
189,Entrada,2021-07-06,Transferência - Liquidação,FII,BTLG11,BTG PACTUAL LOGISTICA FDO INV IMOB FII (BTLG11),EASYNVEST - TITULO CV S/A,16,109.49,1751.84
197,Entrada,2021-07-23,Rendimento,FII,BTLG11,BTG PACTUAL LOGISTICA FDO INV IMOB FII (BTLG11),EASYNVEST - TITULO CV S/A,16,0.70,11.20
205,Entrada,2021-08-13,Rendimento,FII,BTLG11,BTG PACTUAL LOGISTICA FUNDO DE INVESTIMENTO IM...,EASYNVEST - TITULO CV S/A,16,0.74,11.84
212,Entrada,2021-09-24,Rendimento,FII,BTLG11,BTG PACTUAL LOGISTICA FUNDO DE INVESTIMENTO IM...,NU INVEST CORRETORA DE VALORES S.A.,16,0.72,11.52
218,Entrada,2021-10-25,Rendimento,FII,BTLG11,BTG PACTUAL LOGISTICA FUNDO DE INVESTIMENTO IM...,NU INVEST CORRETORA DE VALORES S.A.,16,0.72,11.52
223,Entrada,2021-11-25,Rendimento,FII,BTLG11,BTG PACTUAL LOGISTICA FUNDO DE INVESTIMENTO IM...,NU INVEST CORRETORA DE VALORES S.A.,16,0.72,11.52
233,Entrada,2021-12-23,Rendimento,FII,BTLG11,BTG PACTUAL LOGISTICA FUNDO DE INVESTIMENTO IM...,NU INVEST CORRETORA DE VALORES S.A.,16,0.72,11.52
238,Entrada,2022-01-25,Rendimento,FII,BTLG11,BTG PACTUAL LOGISTICA FUNDO DE INVESTIMENTO IM...,NU INVEST CORRETORA DE VALORES S.A.,16,0.72,11.52
240,Entrada,2022-02-25,Rendimento,FII,BTLG11,BTG PACTUAL LOGISTICA FUNDO DE INVESTIMENTO IM...,NU INVEST CORRETORA DE VALORES S.A.,16,0.72,11.52
247,Entrada,2022-03-22,Rendimento,FII,BTLG11,BTG PACTUAL LOGISTICA FUNDO DE INVESTIMENTO IM...,NU INVEST CORRETORA DE VALORES S.A.,16,0.72,11.52


In [102]:
wallet.print_assets()

Ticker: BIDI4
Nome do produto: BANCO INTER S.A. (BIDI4)
Tipo: ACAO
Quantidade: 0
Valor: 1632.8
Preço médio: 408.20
Rendimentos: 0.00
------------------------------------------------------------
Ticker: MGLU3
Nome do produto: MAGAZINE LUIZA S.A. (MGLU3)
Tipo: ACAO
Quantidade: 700
Valor: 15756.940000000002
Preço médio: 22.51
Rendimentos: 6.12
------------------------------------------------------------
Ticker: VVAR3
Nome do produto: VIA VAREJO S.A. (VVAR3)
Tipo: ACAO
Quantidade: 0
Valor: 6900.659999999999
Preço médio: 27.71
Rendimentos: 0.00
------------------------------------------------------------
Ticker: SPTW11
Nome do produto: SP DOWNTOWN FDO INV IMOB FII (SPTW11)
Tipo: FII
Quantidade: 0
Valor: 1687.88
Preço médio: 84.39
Rendimentos: 328.65
------------------------------------------------------------
Ticker: XPML11
Nome do produto: XP MALLS FDO INV IMOB FII (XPML11)
Tipo: FII
Quantidade: 0
Valor: 707.4
Preço médio: 141.48
Rendimentos: 11.17
-----------------------------------------